In [13]:
using DynamicPolynomials, SparseArrays

println("***Problem setting***")

n=20

println("Number of variable: n=",n)
println("====================")

@ncpolyvar x[1:n]# variables

function star_algebra(mom)
    if mom==1
        return mom
    else
        ind=mom.z .>0
        vars=mom.vars[ind]
        exp=mom.z[ind]
        return prod(vars[i]^exp[i] for i in length(exp):-1:1)
    end
end

function generate_random_poly(v)
    lv=length(v)
    c=2*rand(Float64,lv).-1
    return sum(c[j]*v[j] for j in 1:lv)
end
# random quadratic objective function f
v=reverse(monomials(x,2))
v+=star_algebra.(v)
v=v./2
f=generate_random_poly(v)


# unit sphere constraint
g=[1.0-sum(x.^2)] #type of coefficients of each polynomial must be float

m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

l=ceil(Int64, n/4)

h=Vector{Polynomial{false,Float64}}(undef,l)
randx=2*rand(n).-1# create a feasible solution
randx=rand(1)[1]*randx./sqrt(sum(randx.^2))


for j in 1:l
    h[j]=generate_random_poly(v[2:end])
    h[j]-=h[j](x => randx) #make constraints feasible
end

l=length(h)
println("Number of equality constraints: l=",l)


***Problem setting***
Number of variable: n=20
Number of inequality constraints: m=1
Number of equality constraints: l=5


In [14]:
include("../src/ctpNCPOP.jl")
using .ctpNCPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpNCPOP.get_info(x,f,g,h);

In [15]:
k=2

println("Relaxed order: k=",k)
println("====================")
t=1
println("Sparse order: t=",t)

Relaxed order: k=2
Sparse order: t=1


In [16]:
using NCTSSOS


@time begin
j=1
opt,data=nctssos_first(Vector{Vector{Vector{UInt16}}}([[supp_f];supp_g;supp_h]),[[coe_f];coe_g;coe_h],n,k,[dg;dh],numeq=l,reducebasis=false,obj="eigen",merge=false,TS="block",QUIET=false)
while j <= t-1
    opt,data=nctssos_higher!(NCTSSOS.cdata_type(data),TS="block")
end
end



------------------------------------------------------
The sizes of blocks:
[401, 20]
[1, 1]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 80411           
  Cones                  : 0               
  Scalar variables       : 1057            
  Matrix variables       : 3               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.01            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.07  

InterruptException: InterruptException:

In [17]:
include("../src/ctpNCPOP.jl")
using .ctpNCPOP

opt_val=ctpNCPOP.POP_TS_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,t,
                         tol=1e-3,use_eqcons_to_get_constant_trace=false)

  Computing constant trace status: OPTIMAL
  Constant trace: ak = 3.0
  Number of blocks: omega=

4
  Size of the largest block: s^max=401
  Number of equality trace constraints: zeta=1667
Modeling time:
  4.467298 seconds (16.91 M allocations: 5.729 GiB, 18.26% gc time)
iter=1   val=-10.275754800273232   gap=1.5753773213986577e-16   feas=2.6149467005057776
iter=2   val=-3.4252516000910775   gap=0.7733440327558271   feas=0.9817046808309702
iter=4   val=-3.639475536598281   gap=0.6405369657704308   feas=0.5949528452027811
iter=8   val=-4.126497015484076   gap=0.27277731450604076   feas=0.43504488788325907
iter=16   val=-3.3979608280460645   gap=0.33446632964555484   feas=0.211225698251123
iter=32   val=-3.2229318135313587   gap=0.3215588931236793   feas=0.1129381348688042
iter=64   val=-3.3678969255276656   gap=0.11554459282286064   feas=0.07385159942264946
iter=128   val=-3.276286580559314   gap=0.03094024693833254   feas=0.036431286262451984
iter=256   val=-3.1415561155200367   gap=0.061027381387416715   feas=0.015588266311423342
iter=512   val=-3.129075753931881   gap=0.112276526

-3.134982200629076

In [18]:
#using ctpPOP

#optval=ctpPOP.POP_NLP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)